# Modelo 2030 - Conversión de script a Notebook
Este notebook contiene la conversión del script `scene_2030.py` a celdas Jupyter organizadas: imports, carga de datos, parámetros, definición del modelo Pyomo, resolución y resultados.

Asegúrate de tener instaladas las dependencias: `pandas`, `pyomo`, y el solver `HiGHS` (o ajustar el solver si no está disponible).

## 1. Importar Librerías y Configuración Inicial

In [10]:
# Imports
import pandas as pd
import highspy as hgs
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Parámetros de configuración
tolerancia = 0.00001
perdida = 0.04

## 2. Cargar y Procesar Datos de Centrales Eléctricas

In [11]:
# Cargar archivos CSV (asegúrate que los archivos existan en el mismo directorio)
df_centrales_ex = pd.read_csv('centrales_ex.csv')
df_centrales_ex = df_centrales_ex.fillna(0)

df_centrales_nuevas = pd.read_csv('centrales_n.csv')
df_centrales_nuevas = df_centrales_nuevas.fillna(0)

# Revisar las primeras filas
#display(df_centrales_ex.head())
#display(df_centrales_nuevas.head())

## 3. Configurar Índice de Parámetros de Centrales

In [12]:
# Preparar parámetros (index por nombre de planta)
param_centrales = df_centrales_ex.set_index(['planta_n'])
param_centrales_nuevas = df_centrales_nuevas.set_index(['planta_n'])

## 4. Definir Parámetros del Sistema y Constantes

In [13]:
# Tipos de Centrales
t_centrales = ['biomasa', 'carbon','cc-gnl', 'petroleo_diesel', 'hidro', 'minihidro','eolica','solar', 'geotermia']
t_ernc = ['eolica','solar', 'geotermia','minihidro','hidro' ]
dispnibilidad_hidro = [0.8215,0.6297,0.561]
costo_falla = 500  # $/MWh
year = 2030 - 2016

## 5. Definir Índices de Plantas y Bloques de Carga

In [14]:
index_plantas = param_centrales.index.tolist()
index_plantas_nuevas = param_centrales_nuevas.index.tolist()  # demanda en MW (en la pasada tenia 12k en vez de 1200 xd)
dic_bloques = {'bloque_1': {'duracion': 1200 , 'demanda' : 10233.87729},
               'bloque_2': {'duracion': 4152 , 'demanda' : 7872.0103}, 
               'bloque_3': {'duracion': 3408 , 'demanda' : 6297.872136}}

# Mostrar resumen
print('Plantillas existentes:', index_plantas[:5])
print('Plantillas nuevas:', index_plantas_nuevas[:5])

Plantillas existentes: ['planta_1', 'planta_2', 'planta_3', 'planta_4', 'planta_5']
Plantillas nuevas: ['planta_1', 'planta_2', 'planta_3', 'planta_4', 'planta_5']


## 6. Crear Modelo y Definir Conjuntos | Parámetros | Variables

In [15]:
# Construcción del modelo Pyomo
model = pyo.ConcreteModel()

# Conjuntos
model.CENTRALES = pyo.Set(initialize=index_plantas)
model.CENTRALES_NUEVAS = pyo.Set(initialize=index_plantas_nuevas)
model.BLOQUES = pyo.Set(initialize=['bloque_1','bloque_2','bloque_3'])

# Parámetros en formato dict (Pyomo Any para permitir dicts anidados)
model.param_centrales = pyo.Param(model.CENTRALES, initialize=param_centrales.to_dict(orient='index'), within=pyo.Any)
model.param_centrales_nuevas = pyo.Param(model.CENTRALES_NUEVAS, initialize=param_centrales_nuevas.to_dict(orient='index'), within=pyo.Any)
model.param_bloques = pyo.Param(model.BLOQUES, initialize=dic_bloques, within=pyo.Any)

# Variables(todas las generaciones son en GWh y la potencia en MW)
model.generacion_ex = pyo.Var(model.CENTRALES, model.BLOQUES, within=pyo.NonNegativeReals)
model.generacion_nuevas = pyo.Var(model.CENTRALES_NUEVAS, model.BLOQUES, within=pyo.NonNegativeReals)
model.potencia_in_nuevas = pyo.Var(model.CENTRALES_NUEVAS, within=pyo.NonNegativeReals)
model.falla = pyo.Var(model.BLOQUES, within=pyo.NonNegativeReals)

## 7. Definir Funciones de Restricciones

In [16]:
# Restricciones
def fd_hidro(bloque):
    if bloque == 'bloque_1':
        return dispnibilidad_hidro[0]
    elif bloque == 'bloque_2':
        return dispnibilidad_hidro[1]
    else:
        return dispnibilidad_hidro[2]

def balance_demanda(model, bloque):
    gen_ex = sum(model.generacion_ex[planta, bloque] for planta in model.CENTRALES)
    gen_new = sum(model.generacion_nuevas[planta, bloque] for planta in model.CENTRALES_NUEVAS)
    return (gen_ex + gen_new + model.falla[bloque]) * (1000/(1+perdida)) >= model.param_bloques[bloque]['demanda'] * model.param_bloques[bloque]['duracion']
                                                    # el 1000 es para convertir de GWh a MWh

def max_gen_ex(model, planta, bloque):
    tec = model.param_centrales[planta]['tecnologia']
    efi = 1.0
    if tec in ['hidro', 'hidro_conv', 'minihidro']:
        disp = fd_hidro(bloque)
    else:
        disp = model.param_centrales[planta]['disponibilidad']
        efi = model.param_centrales[planta]['eficiencia'] or 1.0
    # generacion está en GWh (multiplicamos por 1000  para hacer el cambio a MWh), potencia_neta_mw en MW, duracion en horas
    return model.generacion_ex[planta, bloque]*1000 <= model.param_centrales[planta]['potencia_neta_mw'] * model.param_bloques[bloque]['duracion'] * disp

def max_gen_nuevas(model, planta, bloque):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    efi = 1.0
    if tec in ['hidro', 'minihidro']:
        disp = fd_hidro(bloque)
    else:
        disp = model.param_centrales_nuevas[planta]['disponibilidad']
        efi = model.param_centrales_nuevas[planta]['eficiencia'] or 1.0
    # generacion está en GWh (multiplicamos por 1000  para hacer el cambio a MWh), potencia_neta_mw en MW, duracion en horas
    return model.generacion_nuevas[planta, bloque]*1000 <= model.potencia_in_nuevas[planta] * model.param_bloques[bloque]['duracion'] * disp

def max_capacidad_nuevas(model, planta):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    if tec in t_ernc:
        limite = model.param_centrales_nuevas[planta]['maxima_restriccion_2030_MW']
        return model.potencia_in_nuevas[planta] <= limite if limite > 0 else pyo.Constraint.Skip # Si el límite es 0, no hay restricción
    else:
        return pyo.Constraint.Skip

def anualidad(r, n): # r es tasa de descuento, n es vida util en años
    return r / (1 - (1 + r)**(-n))

# Adjuntar restricciones
model.demanda_constraint = pyo.Constraint(model.BLOQUES, rule=balance_demanda)
model.max_gen_constraint = pyo.Constraint(model.CENTRALES, model.BLOQUES, rule=max_gen_ex)
model.max_gen_nuevas_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, rule=max_gen_nuevas)
model.max_capacidad_nuevas_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, rule=max_capacidad_nuevas)

## 9. Definir Expresiones de Costos y Función Objetivo

In [17]:
# Función objetivo (expresiones)

# Costo operación (costos variables) 2030
model.op_ex = pyo.Expression(expr=sum(model.generacion_ex[planta, bloque] * 1000 * # pasamos a MWh
                                (model.param_centrales[planta]['costo_variable_nc']
                                + model.param_centrales[planta]['costo_variable_t']
                                + model.param_centrales[planta]['costo_var_comb_16_usd_mwh']) 
                                for planta in model.CENTRALES 
                                for bloque in model.BLOQUES)) # esto queda en USD

model.op_new = pyo.Expression(expr=sum(model.generacion_nuevas[planta, bloque] * 1000 * # pasamos a MWh
                                (model.param_centrales_nuevas[planta]['cvnc_usd_MWh'] 
                                + model.param_centrales_nuevas[planta]['linea_peaje_usd_MWh']
                                + model.param_centrales_nuevas[planta]['costo_var_comb_16_usd_mwh'])  
                                for planta in model.CENTRALES_NUEVAS 
                                for bloque in model.BLOQUES)) # esto queda en USD

# Costo inversión (anualidad) 2030
model.inv_new = pyo.Expression(expr=sum(model.potencia_in_nuevas[planta]* 
                                        anualidad(model.param_centrales_nuevas[planta]['tasa_descuento'], model.param_centrales_nuevas[planta]['vida_util_anos']) * 
                                        model.param_centrales_nuevas[planta]['inversion_usd_kW_neto']* 1000 # pasamos a MW
                                        for planta in model.CENTRALES_NUEVAS)) # esto queda en USD/año

# costo FALLAS (recordar que falla está en GWh)
model.costo_fallas = pyo.Expression(expr=sum(model.falla[bloque] * 1000 * # pasamos a MWh
                            costo_falla for bloque in model.BLOQUES))

r_df = 0.01
df_2016_2030 = 1 / (1 + r_df)**(year)

# minimizar (costo operacion + inversion + costo falla)
model.obj = pyo.Objective(expr = df_2016_2030 * (model.op_ex + model.op_new + model.inv_new + model.costo_fallas), sense = pyo.minimize)

## 10. Resolver el Modelo de Optimización

In [18]:
# Resolver el modelo
solver = pyo.SolverFactory('highs')
solver.options['mip_rel_gap'] = tolerancia
results = solver.solve(model, tee=True)
print(f"Status: {results}")

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


LP   has 131 rows; 146 cols; 314 nonzeros
Coefficient ranges:
  Matrix [1e+00, 4e+03]
  Cost   [9e+03, 4e+05]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 3e+07]
Presolving model
63 rows, 143 cols, 243 nonzeros  0s
63 rows, 140 cols, 240 nonzeros  0s
Presolve : Reductions: rows 63(-68); columns 140(-6); elements 240(-74)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(4.3901e+07) 0s
         40     1.5338548176e+09 Pr: 0(0); Du: 0(1.42109e-14) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 40
Objective value     :  1.5338548176e+09
P-D objective error :  1.5543751357e-16
HiGHS run time      :          0.01
Status: 
Problem: 
- Lower bound: 1533854817.584971
  Upper bound: 1533854817.584971
  Number of objectives: 1
  Number of constraints: nan
  Number of variables: nan
  Sense: minimize
Solver: 
- Status: ok
  Te

## 11. Análisis de Resultados por Tipo de Tecnología

In [19]:
# ver resultados por tipo de central
for tec in t_centrales:
    gen_tec = sum(model.generacion_ex[planta, bloque].value 
                  for planta in model.CENTRALES if model.param_centrales[planta]['tecnologia'] == tec
                  for bloque in model.BLOQUES)
    gen_tec_nuevas = sum(model.generacion_nuevas[planta, bloque].value 
                  for planta in model.CENTRALES_NUEVAS if model.param_centrales_nuevas[planta]['tecnologia'] == tec
                  for bloque in model.BLOQUES)
    gen_tec += gen_tec_nuevas
    print(f'Generación total de tecnología {tec}: {gen_tec} GWh')

Generación total de tecnología biomasa: 1271.6352000000002 GWh
Generación total de tecnología carbon: 34581.53984929151 GWh
Generación total de tecnología cc-gnl: 815.796 GWh
Generación total de tecnología petroleo_diesel: 849.5926435200009 GWh
Generación total de tecnología hidro: 30708.4795704 GWh
Generación total de tecnología minihidro: 0.0 GWh
Generación total de tecnología eolica: 420.48 GWh
Generación total de tecnología solar: 438.0 GWh
Generación total de tecnología geotermia: 0.0 GWh


## 12. Análisis de Generación Total del Sistema y Balance de Carga

In [20]:
#sumatoria todas las generaciones
total = 0
for bloque in model.BLOQUES:
    gen_ex_bloque = sum(model.generacion_ex[planta, bloque].value for planta in model.CENTRALES)
    gen_new_bloque = sum(model.generacion_nuevas[planta, bloque].value for planta in model.CENTRALES_NUEVAS)
    falla_bloque = model.falla[bloque].value
    print(f'Generación total en {bloque}: {gen_ex_bloque+gen_new_bloque} GWh, Falla: {falla_bloque} GWh')
    total += gen_ex_bloque + gen_new_bloque + falla_bloque

print(f'Generación total en el sistema (incluyendo fallas): {total} GWh')

Generación total en bloque_1: 12771.87885792 GWh, Falla: 0.0 GWh
Generación total en bloque_2: 33991.970236224 GWh, Falla: 0.0 GWh
Generación total en bloque_3: 22321.67416906752 GWh, Falla: 0.0 GWh
Generación total en el sistema (incluyendo fallas): 69085.52326321151 GWh
